In [2]:
from neo4j import GraphDatabase
from py2neo import Graph
import pandas as pd
import numpy as np

In [97]:
uri="neo4j+s://1c8f07f5.databases.neo4j.io"
username="neo4j"
password="uycJUUxlA1SDUncKIgImCp_DT8hASOiqSQ-V7pnLRCY"

In [3]:
tmdb = pd.read_csv("data\\tmdb_movies_data.csv")
tmdb["cast"] = tmdb["cast"].apply(lambda x: str(x))
tmdb["cast"] = tmdb["cast"].apply(lambda x: x.split("|"))
tmdb["genres"] = tmdb["genres"].apply(lambda x: str(x))
tmdb["genres"] = tmdb["genres"].apply(lambda x: x.split("|"))
tmdb = tmdb[["original_title", "director","cast", "genres", "release_date", "popularity"]]
tmdb = tmdb.dropna()

In [105]:
# Connect to the Neo4j database
graph = GraphDatabase.driver(uri, auth=(username, password))

# Create a session
session = graph.session()

# Iterate over the DataFrame rows
for _, row in tmdb.iterrows():
    # Create Movie node and Director node with DIRECTED relationship
    query_create_movie_and_director = """
    MERGE (m:Movie {
        original_title: $original_title,
        release_date: $release_date,
        popularity: $popularity
    })
    MERGE (d:Director {name: $director})
    MERGE (d)-[:DIRECTED]->(m)
    """
    parameters = {
        "original_title": row['original_title'],
        "director": row['director'],
        "release_date": row['release_date'],
        "popularity": row['popularity']
    }
    session.run(query_create_movie_and_director, parameters)

     # Create Actor nodes and ACTED_IN relationships
    query_create_actor = """
    MERGE (a:Actor {name: $actor})
    WITH a
    MATCH (m:Movie {original_title: $original_title, release_date: $release_date, popularity: $popularity})
    MERGE (a)-[:ACTED_IN]->(m)
    """
    for actor in row['cast']:
        parameters_actor = {
            "original_title": row['original_title'],
            "director": row['director'],
            "release_date": row['release_date'],
            "popularity": row['popularity'],
            "actor": actor
        }
        session.run(query_create_actor, parameters_actor)

    # Create Genre nodes and IN_GENRE relationships
    query_create_genre = """
    MERGE (g:Genre {name: $genre})
    WITH g
    MATCH (m:Movie {original_title: $original_title, release_date: $release_date, popularity: $popularity})
    MERGE (m)-[:IN_GENRE]->(g)
    """
    for genre in row['genres']:
        parameters_genre = {
            "original_title": row['original_title'],
            "director": row['director'],
            "release_date": row['release_date'],
            "popularity": row['popularity'],
            "genre": genre
        }
        session.run(query_create_genre, parameters_genre)

        
# Close the session
session.close()

# Close the Neo4j connection
graph.close()